# Merging state, county, and city polygons with political parties

In [ ]:
import ibis
from ibis import _
import streamlit as st
import ibis.expr.datatypes as dt  # Make sure to import the necessary module

conn = ibis.duckdb.connect(extensions=["spatial"])

# State 

In [ ]:
#getting party
state = (conn
        .read_csv("1976-2020-president.csv")
        # .filter(_. year >=2000)
        .rename(state="state_po" , party = "party_simplified")  # rename columns
        .group_by(["year", "state"])
        .aggregate(party=_.party.argmax(_.candidatevotes))  # winning party 
        .select("year", "state", "party")  # select only relevant columns
        )

In [ ]:
# merging with state polygons
state_boundaries = "https://data.source.coop/cboettig/us-boundaries/us-state-territory.parquet"

df_state = (conn
            .read_parquet(state_boundaries)
            .rename(state = "STUSPS", state_ = "NAME")
            .select("state","geometry")
            .join(state,"state",how = "inner")
            .mutate(county = None)
            .mutate(municipal = None)
            .mutate(jurisdiction = ibis.literal("State"))
            .cast({"geometry": "geometry","county":"string","municipal": "string"})
            .select("state", "county", "municipal","jurisdiction","geometry", "year", "party")
            )

# County

In [ ]:
# getting party
county = (conn
        .read_csv("countypres_2000-2020.csv")
        .filter((_.totalvotes > 0))  # filter empty votes
        .rename(state="state_po", state_name = "state") 
        .mutate(county = _.county_name + ibis.literal(" COUNTY"))
        .group_by(["year", "state", "county", "state_name", "party"])
        .aggregate(
            total_candidate_votes=_.candidatevotes.sum()  #getting total votes per candidate 
        )
        .group_by(["year", "state", "county", "state_name"])
        .aggregate(
            party=_.party.argmax(_.total_candidate_votes)  # party with the highest total votes
        )
        .select("year", "state", "county", "party","state_name")  
)

In [ ]:
# merging with county polygons
county_boundaries = "https://data.source.coop/cboettig/us-boundaries/us-county.parquet"
df_county = (conn
        .read_parquet(county_boundaries)
        .mutate(county = _.NAMELSAD.upper(), state_name = _.STATE_NAME.upper())
        .select("state_name","county","geometry")
        .join(county,["state_name","county"],how = "inner")
        .mutate(municipal = None)
        .cast({"geometry": "geometry","municipal": "string"})
        .mutate(jurisdiction = ibis.literal("County"))
        .select("state", "county", "municipal","jurisdiction","geometry", "year", "party")
        )


# Getting Municipals 

In [ ]:
localities_boundaries = "us_localities.parquet"
locality = (conn
            .read_parquet(localities_boundaries)
            .mutate(county = _.county.upper())
            .mutate(municipal = _.municipal.upper())
           )

df_city = (county
            .drop("state_name")
            .join(locality, ["state","county"], how = "inner")
            .cast({"geometry": "geometry"})
            .mutate(jurisdiction = ibis.literal("Municipal"))
            .select("state", "county", "municipal","jurisdiction","geometry", "year", "party")
            )


# Make PMTiles with only state/county. Each jurisdiction type is its own layer. 

In [ ]:
import subprocess
import os
from huggingface_hub import HfApi, login
import streamlit as st

login(st.secrets["HF_TOKEN"])
# api = HfApi(add_to_git_credential=False)
api = HfApi()

def hf_upload(file, repo_id):
    info = api.upload_file(
            path_or_fileobj=file,
            path_in_repo=file,
            repo_id=repo_id,
            repo_type="dataset",
        )
def generate_pmtiles(input_file, input_file2, output_file, max_zoom=12):
    # Ensure Tippecanoe is installed
    if subprocess.call(["which", "tippecanoe"], stdout=subprocess.DEVNULL) != 0:
        raise RuntimeError("Tippecanoe is not installed or not in PATH")

    # Construct the Tippecanoe command
    command = [
        "tippecanoe",
        "-o", output_file,
        "-zg",
        "--extend-zooms-if-still-dropping",
        "--force",
        "--projection", "EPSG:4326",  
        "-L","state:"+input_file,
        "-L","county:"+input_file2
    ]
    # Run Tippecanoe
    try:
        subprocess.run(command, check=True)
        print(f"Successfully generated PMTiles file: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error running Tippecanoe: {e}")


In [ ]:
gdf_state = df_state.execute().set_crs("EPSG:4326")
gdf_state.to_file("party_state.geojson")

gdf_county = df_county.execute().set_crs("EPSG:4326")
gdf_county.to_file("party_county.geojson")

# city data too large to add to pmtiles :( 
# gdf_city = df_city.execute().set_crs("EPSG:4326")
# gdf_city.to_file("party_municipal.geojson")

generate_pmtiles("party_state.geojson", "party_county.geojson", "party_polygons.pmtiles")
hf_upload("party_polygons.pmtiles", "boettiger-lab/landvote")


# Combine all 3 jurisdiction types into a parquet.

In [ ]:
df_temp = df_county.union(df_city)
df = df_temp.union(df_state)
df.execute().set_crs("EPSG:4326").to_parquet("party_polygons.parquet")
hf_upload("party_polygons.parquet", "boettiger-lab/landvote")
